In [1]:
import pandas as pd
import time
import xml.etree.ElementTree as ET
import requests

In [2]:
## Read in abstracts_final.csv as df_abstracts
df_abstracts = pd.read_csv('abstracts_final.csv')

In [3]:
## Exclude abstracts from 2024
df_abstracts_subset = df_abstracts[df_abstracts['year'] != 2024]

In [4]:
## Define a new column called first_author clean that is in Lastname, First Initial
df_abstracts_subset['first_author_clean'] = df_abstracts_subset['first_author'].str.split().str[-1] + ', ' + df_abstracts_subset['first_author'].str.split().str[0].str[0]

## Remove commas from the first_author_clean column
df_abstracts_subset['first_author_clean'] = df_abstracts_subset['first_author_clean'].str.replace(',','')

## Remove periods from the first_author_clean column
df_abstracts_subset['first_author_clean'] = df_abstracts_subset['first_author_clean'].str.replace('.','')

## Make abstract_title lowercase
df_abstracts_subset['abstract_title_lower'] = df_abstracts_subset['abstract_title'].str.lower()

## Replace - with space in abstract_title_lower
df_abstracts_subset['abstract_title_lower'] = df_abstracts_subset['abstract_title_lower'].str.replace('-',' ')

## Remove punctuation and special characters from abstract_title_lower
df_abstracts_subset['abstract_title_lower'] = df_abstracts_subset['abstract_title_lower'].str.replace(r'[^\w\s]', ' ', regex=True)

# Remove filler words from abstract_title_lower
# can you rewrite the above in a cleaner way using a list
filler_words = ['the','a','an','of','and','in','for','on','to','with','by','from','at','as','into','over']

## Remove filler words from abstract_title_lower
df_abstracts_subset['abstract_title_lower'] = df_abstracts_subset['abstract_title_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in (filler_words)]))

## Create new column that is the first 5 words in df_abstracts_subset['abstract_title_lower']
df_abstracts_subset['abstract_title_short'] = df_abstracts_subset['abstract_title_lower'].str.split().str[:5].str.join(' ')

df_abstracts_subset.head()

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59178/134107155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstracts_subset['first_author_clean'] = df_abstracts_subset['first_author'].str.split().str[-1] + ', ' + df_abstracts_subset['first_author'].str.split().str[0].str[0]
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59178/134107155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstracts_subset['first_author_clean'] = df_abstracts_subset['first_author_clean'].s

,Unnamed: 0,control_number,session_number,session_type,abstract_title,author_block,abstract_body,abstract_type,abstract_scientific_area,abstract_clinical_area,year,author_list,first_author,senior_author,institution_list,smart_institution5,first_author_clean,abstract_title_lower,abstract_title_short
0,0,ASC20161024,1.01,QuickShot,Mimetic sHDL Nanoparticles: a Novel Drug-Deliv...,"P. T. White<sup>1</sup>, C. Subramanian<sup>1<...",<strong>Introduction</strong>: Triple negativ...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,"['P. T. White<sup>1</sup>', ' C. Subramanian<s...",P. White,M. Cohen,"['University Of Michigan,Department Of Biomedi...",University Of Michigan,White P,mimetic shdl nanoparticles novel drug delivery...,mimetic shdl nanoparticles novel drug
1,1,ASC20160315,1.02,QuickShot,Novel Nano-Liposomes for Multi-Modal Image-Gui...,"P. Patel<sup>1</sup>, T. Kato<sup>1</sup>, H. ...",<strong>Introduction</strong>: To investigate ...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,"['P. Patel<sup>1</sup>', ' T. Kato<sup>1</sup>...",P. Patel,K. Yasufuku,"['University of Toronto,Division Of Thoracic S...",University Of Toronto,Patel P,novel nano liposomes multi modal image guided ...,novel nano liposomes multi modal
2,2,ASC20160574,1.03,QuickShot,Tumor-Targeting Nanotheranostic Micelles for N...,"R. Jaskula Sztul<sup>1,3</sup>, G. Chen<sup>2<...",<strong>Introduction</strong>: Although neuro...,Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,"['R. Jaskula Sztul<sup>1', '3</sup>', ' G. Che...",R. Jaskula Sztul,H. Chen,"['University Of Wisconsin,Surgery,Madison, WI,...",University Of Wisconsin,Sztul R,tumor targeting nanotheranostic micelles neuro...,tumor targeting nanotheranostic micelles neuro...
3,3,ASC20160105,1.04,QuickShot,SDF-1&alpha; Decreases Inflammation in Diabeti...,"M. M. Hodges<sup>1</sup>, C. Zgheib<sup>1</sup...","<strong>Introduction</strong>: In 2014, the U...",Basic Science,Experimental Therapeutics/Other,Cross Disciplinary,2016,"['M. M. Hodges<sup>1</sup>', ' C. Zgheib<sup>1...",M. Hodges,K. Liechty,"['University Of Colorado Denver,Laboratory For...",University Of Colorado Denver,Hodges M,sdf 1 alpha decreases inflammation diabetic de...,sdf 1 alpha decreases inflammation
4,4,ASC20161100,1.05,QuickShot,Mechanistic Analysis: Alcohol Induces Apoptosi...,"I. J. Lawandy<sup>1</sup>, B. A. Potz<sup>1</s...",<strong>Introduction</strong>: Autophagy serv...,Basic Science,Disease Pathogenesis,General Surgery,2016,"['I. J. Lawandy<sup>1</sup>', ' B. A. Potz<sup...",I. Lawandy,F. Sellke,"['Brown University,Surgery/Cardiothoracic Surg...",Brown University School Of Medicine,Lawandy I,mechanistic analysis alcohol induces apoptosis...,mechanistic analysis alcohol induces apoptosis


In [5]:
def esearch_query(payload, retmax = 100000, sleep=0.11):
    """
    Query the esearch E-utility.
    """
    url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi&api_key=cabe196b79f90e2c37ff374214703ad6ce08'
    payload['retmax'] = retmax
    payload['retstart'] = 0
    ids = list()
    count = 1
    while payload['retstart'] < count:
        response = requests.get(url, params=payload)
        # print(response.content)
        xml = ET.fromstring(response.content)
        count = int(xml.findtext('Count'))
        ids += [xml_id.text for xml_id in xml.findall('IdList/Id')]
        payload['retstart'] += retmax
        time.sleep(sleep)
    print(ids)
    return ids


In [6]:
## Split df_abstracts_subset into rows of 100 elements each plus any leftover

## We want each sub-dataframe to be 100 rows long
n = 100

## Split the dataframe into sub-dataframes
df_abstracts_subset_split = [df_abstracts_subset[i:i+n] for i in range(0,df_abstracts_subset.shape[0],n)]

In [7]:
for i in range(len(df_abstracts_subset_split)):
    print('************')
    print('SUBSET' + str(i))
    df_abstracts_subset_split[i]['pmids'] = df_abstracts_subset_split[i].apply(
        lambda x: esearch_query({'db': 'pubmed', 
                                 'term':    '{}[Title]'.format(x['abstract_title_short']) + 
                                            ' AND ' + '{}[Author]'.format(x['first_author_clean'] + 
                                            ' AND ' + '({}[Date - Publication] : {}[Date - Publication])'.format(x['year'], '3000')),
                                 'mindate': x['year']}), 
        axis=1
    )


************
SUBSET0
['31371177']
['27584018']
[]
[]
[]
['30301901']
[]
[]
['28546859']
[]
[]
[]
[]
['27884325', '26711625']
['29128180']
[]
[]
[]
['28341269']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [ ]:
## Merge all df_abstracts_subset_split into one dataframe
df_abstracts_subset_merged = pd.concat(df_abstracts_subset_split)

In [ ]:
## Define a new column called first_author clean that is in Lastname, First Initial
df_abstracts_subset_merged['senior_author_clean'] = df_abstracts_subset_merged['senior_author'].str.split().str[-1] + ', ' + df_abstracts_subset_merged['senior_author'].str.split().str[0].str[0]

## Remove commas from the first_author_clean column
df_abstracts_subset_merged['senior_author_clean'] = df_abstracts_subset_merged['senior_author_clean'].str.replace(',','')

## Remove periods from the first_author_clean column
df_abstracts_subset_merged['senior_author_clean'] = df_abstracts_subset_merged['senior_author_clean'].str.replace('.','')

## Create new column that is the first 3 words in df_abstracts_subset['abstract_title_lower']
df_abstracts_subset_merged['abstract_title_shorter'] = df_abstracts_subset['abstract_title_lower'].str.split().str[:3].str.join(' ')

## Create new column that is the last 3 words in df_abstracts_subset['abstract_title_lower']
df_abstracts_subset_merged['abstract_title_end'] = df_abstracts_subset['abstract_title_lower'].str.split().str[-3:].str.join(' ')

## Create a subset dataframe where pmids is empty
df_round2 = df_abstracts_subset_merged[df_abstracts_subset_merged['pmids'].apply(lambda x: len(x) == 0)]

In [ ]:
## Split df_abstracts_subset into rows of 100 elements each plus any leftover

## We want each sub-dataframe to be 100 rows long
n = 100

## Split the dataframe into sub-dataframes
df_round2_split = [df_round2[i:i+n] for i in range(0,df_round2.shape[0],n)]

df_round2_split[0]

len(df_round2_split)

82

In [ ]:
for i in range(len(df_round2_split)):

    if i < 42:
        continue
    
    print('************')
    print('SUBSET' + str(i))

    ## make new column called senior_pmids and apply esearch_query if pmids is empty, otherwise transfer pmids to new column senior_pmids
    df_round2_split[i]['senior_pmids'] = df_round2_split[i].apply(
        lambda x: esearch_query({'db': 'pubmed', 
                                 'term':    '{}[Title]'.format(x['abstract_title_end']) + 
                                            ' AND ' + '{}[Author]'.format(x['first_author_clean'] + 
                                            ' AND ' + '({}[Date - Publication] : {}[Date - Publication])'.format(x['year'], '3000')),
                                 'mindate': x['year']}), 
        axis=1,
    )

************
SUBSET42
[]
[]
[]
[]
[]
['36598757']
[]
['32330729']
['35159029']
[]
[]
['35782393']
[]
[]
['34061473', '33038872']
['32662520']
['38051183', '37801489', '37234524', '35620296', '34357569', '31810383']
[]
[]
['32858321']
['37385476', '34564667']
[]


KeyboardInterrupt: 

In [ ]:


# df_abstracts_subset_merged.to_csv('data_upto_subset_final.csv')

## In df_abstracts_subset_merge abstract type, change 'Outcomes/Clinical' to 'Clinical/Outcomes'
df_abstracts_subset_merged['abstract_type'] = df_abstracts_subset_merged['abstract_type'].str.replace('Outcomes/Clinical', 'Clinical/Outcomes')

## Change df_abstracts_subset_merge session type to lowercase and back to title case
df_abstracts_subset_merged['session_type'] = df_abstracts_subset_merged['session_type'].str.lower().str.title()

## Remove any spaces from session type
df_abstracts_subset_merged['session_type'] = df_abstracts_subset_merged['session_type'].str.replace(' ', '')

## Create a new column called pmids_count that is the length of the pmids column
## If the pmids column is empty, pmids_count is 0
df_abstracts_subset_merged['pmids_count'] = df_abstracts_subset_merged['pmids'].apply(lambda x: len(x) if x else 0)

## Create a new column called has_pmid that is True if pmids_count > 0
df_abstracts_subset_merged['has_pmid'] = df_abstracts_subset_merged['pmids_count'] > 0

## Create a new dataframe that groups by session_type and year.
## Calculate the number of abstracts with pmid and the total number of abstracts

df_abstracts_subset_merged_grouped = df_abstracts_subset_merged.groupby(['session_type','year']).agg(
    num_pmids = ('has_pmid', 'sum'),
    num_abstracts = ('control_number', 'nunique')
)

## Reset the index of df_abstracts_subset_merged_grouped
df_abstracts_subset_merged_grouped.reset_index(inplace=True)

## Calculate the percentage of abstracts with pmid
df_abstracts_subset_merged_grouped['pct_pmids'] = df_abstracts_subset_merged_grouped['num_pmids'] / df_abstracts_subset_merged_grouped['num_abstracts']

## Plot the data in a bargraph use plotly.express

import plotly.express as px

fig = px.bar(
    df_abstracts_subset_merged_grouped[df_abstracts_subset_merged_grouped['session_type'] != 'Poster'],
    x='year',
    y='pct_pmids',
    color='session_type',
    barmode='group'
)

## Make pretty
fig.update_layout(
    title='Percentage of Abstracts with PMID by Year and Session Type',
    xaxis_title='Year',
    yaxis_title='Percentage of Abstracts with PMID',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2000,
        dtick = 1
    )
)

fig.show()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

